| title               | tags      | created | modified |
|---------------------|-----------|---------|----------|
| dev-rag-deepdoc-pdf | [dev,rag] |  Sun Mar 16 03:09:57 CST 2025 | Sun Mar 16 03:09:57 CST 2025  |

## 需求：
> 基于ragflow进行二次开发，对pdf文档添加图片解析

### 1. 图片提取

v1: 尝试使用pdfplumber提取图片，但色域为CMYK的图片保存下来不是rgb通道，使用pillow的convert('GRB')也无效，可能需要加载icc、icm文件进行转换

v2: 遂换成fitz提取图片流，使用pdfplumber提取图片id，再将图片id给到fitz



1


In [ ]:
kkkk